In [49]:
from pdf2image import convert_from_path
from PIL import Image
import cv2
import numpy as np
from PIL import Image
import pytesseract

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns', None)


In [34]:
def convert_pdf_to_image(pdf_path, rotation_angle=0):
    images = convert_from_path(pdf_path, dpi=600)
    print(f"Found {len(images)} pages in the PDF.")

    # Take the first page
    image = images[10]

    # Rotate manually if angle is given
    if rotation_angle != 0:
        print(f"Rotating image by {rotation_angle} degrees...")
        image = image.rotate(rotation_angle, expand=True)

    # Save the result
    image_path = "invoice_page1.png"
    image.save(image_path, "PNG")
    print(f"Saved rotated image to {image_path}")
    return image_path


def preprocess_image(image_path):
    # Load image and convert to grayscale
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to binarize (better for OCR)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return thresh

def extract_text_from_image(image_cv):
    # Convert OpenCV image to PIL format for Tesseract
    pil_image = Image.fromarray(image_cv)
    config = r'--oem 3 --psm 6'
    
    df = pytesseract.image_to_data(pil_image, config=config, lang="spa",output_type=pytesseract.Output.DATAFRAME)

    lines = df.groupby("line_num")["text"].apply(lambda x: " ".join(x)).reset_index(name="line_text")
    return lines




In [35]:
def main():
    pdf_path = "6073376343.pdf"
    convert_pdf_to_image(pdf_path, rotation_angle=90)

    
    image_path = "invoice_page1.png"  # Use your converted image here

    processed_image = preprocess_image(image_path)
    text = extract_text_from_image(processed_image)

    print("===== OCR TEXT =====")
    print(text)

    return text

In [43]:
# text = main()

#### best one

In [46]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
import pandas as pd

# ──────────────────────────────
# Cargar y preprocesar imagen
# ──────────────────────────────

def convert_pdf_to_image(pdf_path, rotation_angle=0):
    images = convert_from_path(pdf_path, dpi=600)
    print(f"Found {len(images)} pages in the PDF.")

    # Take the first page
    image = images[10]

    # Rotate manually if angle is given
    if rotation_angle != 0:
        print(f"Rotating image by {rotation_angle} degrees...")
        image = image.rotate(rotation_angle, expand=True)

    # Save the result
    image_path = "invoice_page1.png"
    image.save(image_path, "PNG")
    print(f"Saved rotated image to {image_path}")
    return image_path

    

# def preprocess_image(image_path):
#     image = cv2.imread(image_path)
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Threshold adaptativo para mejorar OCR
#     thresh = cv2.adaptiveThreshold(
#         gray, 255,
#         cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#         cv2.THRESH_BINARY,
#         11, 2
#     )

    # return thresh

def preprocess_image(image_path):
    # Load image and convert to grayscale
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to binarize (better for OCR)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return thresh
    

# ──────────────────────────────
# OCR con Tesseract y DataFrame
# ──────────────────────────────
def extract_ocr_dataframe(image_cv, lang="spa", min_conf=50):
    pil_image = Image.fromarray(image_cv)

    config = r'--oem 3 --psm 6'

    df = pytesseract.image_to_data(
        pil_image,
        lang=lang,
        config=config,
        output_type=pytesseract.Output.DATAFRAME
    )

    # Limpiar resultados: quitar palabras vacías o poco confiables
    df = df[df.conf != -1]
    # df = df[df.text.notna() & df.text.str.strip().astype(bool)]
    # df = df[df.conf.astype(float) >= min_conf]

    return df.reset_index(drop=True)

# ──────────────────────────────
# Ejecutar OCR desde imagen local
# ──────────────────────────────
def main():

    pdf_path = "6073376343.pdf"
    convert_pdf_to_image(pdf_path, rotation_angle=90)

    

    
    image_path = "invoice_page1.png"  # Asegúrate que esta imagen exista

    # processed = preprocess_image(image_path)
    processed_image = preprocess_image(image_path)
    df = extract_ocr_dataframe(processed)

    print("📄 Texto detectado:")
    print(df[['text', 'conf', 'left', 'top']])

    # Agrupar por línea (opcional)
    lines = df.groupby("line_num")["text"].apply(lambda x: " ".join(x)).reset_index(name="line_text")
    print("\n🧾 Líneas agrupadas:")

    
    lines.to_csv('output.csv', header=True, index=False)
    
    return lines

In [47]:
df = main()

Found 12 pages in the PDF.
Rotating image by 90 degrees...
Saved rotated image to invoice_page1.png
📄 Texto detectado:
             text       conf  left   top
0      COBRE.EN-—   0.000000   404   324
1        UA/10/24   0.000000   726   324
2    EN-15:43:18'   0.000000  1004   323
3           JAti-  24.106094  2574   343
4            MART  26.501480  2817   346
..            ...        ...   ...   ...
329             1  41.467545  3405  3637
330             a  35.229942  6332  3784
331           na*   0.000000   336  4813
332             j   0.000000  6551  4837
333             Í  30.277237  6529  4905

[334 rows x 4 columns]

🧾 Líneas agrupadas:


In [50]:
df

,line_num,line_text
0,1,COBRE.EN-— UA/10/24 EN-15:43:18' JAti- MART ST...
1,2,PARA — PERIOD TERMINA 05/10/24 DETALLE RECIBOS...
2,3,a - (COPIA PROV
3,4,1 o POT
4,5,4 e | Ñ Expid. £ Fuctur:
5,6,1D Prov; 50005 DIST ARCA CONTINENTAL SRL CV P....
6,7,Div: 28 Dept 95 ) O (Rovf 1 de 1 recibos)
7,8,ID“Transpt: VNDR LOCAL VENDOR Cajas Recibidas:...
8,9,Arte UPC hi Stock + Cant Pag Costo Pag e Cant ...
9,10,Descripcion Tama*o Color $IEPS IEPS “RIVA : TY...


In [ ]:
text

In [9]:
lines_df

,line_num,line_text
0,1,CORRE EN 15:63:18 18N< “TIENDA $. isp2204 $. ....
1,2,"PARA —— FIRIOD TERMINA 06/10/24 ""DETALLE RECIB..."
2,3,e PROV 1D Transpt: VNDR LOCAL. VENDOR Cajas Re...
3,4,_ POr UPC Stock * Cant Paq Costo Paq “Cant Fec...
4,5,Expid. £ Fuctur. Descripcion Color: $IEPS IEPS...
5,6,FUZE TEA 600M NEGRO 600 VERDE 758.16 COSTO TOTAL:
6,7,"Cost de Compra: 89,164,68"
7,8,"COSTO TOTAL ORDEN DE COMPRA; 102,507.98"
8,9,"Diferencia en costo de Oc: 7,492,24-"
9,10,* = Tag Seguridad Prov.
